In [2]:
# Import the API key from api_config.py
import pandas as pd
from api_config import *
import requests

# Define the API URL and parameters
url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"
querystring = {"withFlightInfoOnly": "true"}

# Use the API key from the imported config file
headers = {
    "X-RapidAPI-Key": RAPIDAPI_KEY,
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

# Make the request and print the response
response = requests.get(url, headers=headers, params=querystring)
print(response.text)

{"searchBy":{"lat":52.31,"lon":13.24},"count":1,"items":[{"icao":"EDDB","iata":"BER","name":"Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE","timeZone":"Europe/Berlin"}]}


In [4]:
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'count': 1,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin'}]}

In [6]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889


In [8]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)

  list_for_df = []

  for index, value in enumerate(latitudes):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
    "X-RapidAPI-Key": RAPIDAPI_KEY,
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

In [10]:
# coordinates for Berlin, Paris, London
latitudes = [52.5200, 48.8567, 51.5072]
longitudes = [13.4050, 2.3522, -0.1275]

icao_airport_codes_df = icao_airport_codes(latitudes, longitudes)
icao_airport_codes_df

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,LFPB,LBG,Paris -Le Bourget,-Le Bourget,Paris,FR,Europe/Paris,48.96940,2.441390
2,LFPO,ORY,Paris -Orly,-Orly,Paris,FR,Europe/Paris,48.72530,2.359440
3,LFPG,CDG,Paris Charles de Gaulle,Charles de Gaulle,Paris,FR,Europe/Paris,49.01280,2.549999
4,LFOB,BVA,Beauvais/Tillé Paris Beauvais Tillé,Paris Beauvais Tillé,Beauvais/Tillé,FR,Europe/Paris,49.45440,2.112780
5,EGLC,LCY,London City,City,London,GB,Europe/London,51.50530,0.055277
6,EGLL,LHR,London Heathrow,Heathrow,London,GB,Europe/London,51.47060,-0.461941
7,EGKR,KRH,Redhill Aerodrome,Aerodrome,Redhill,GB,Europe/London,51.21360,-0.138611
8,EGKK,LGW,London Gatwick,Gatwick,London,GB,Europe/London,51.14810,-0.190277
9,EGGW,LTN,London Luton,Luton,London,GB,Europe/London,51.87470,-0.368333


### Prepare a dataframe for pushing to the database

In [12]:
# Selecting only the columns we need
airports_to_db = icao_airport_codes_df[["icao", "name", "municipalityName"]]
airports_to_db

,icao,name,municipalityName
0,EDDB,Berlin Brandenburg,Berlin
1,LFPB,Paris -Le Bourget,Paris
2,LFPO,Paris -Orly,Paris
3,LFPG,Paris Charles de Gaulle,Paris
4,LFOB,Beauvais/Tillé Paris Beauvais Tillé,Beauvais/Tillé
5,EGLC,London City,London
6,EGLL,London Heathrow,London
7,EGKR,Redhill Aerodrome,Redhill
8,EGKK,London Gatwick,London
9,EGGW,London Luton,London


### Read the cities table from the database to fetch the "city_id" column

In [25]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = "forgot1234"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [14]:
schema = "sql_workshop"
host = "34.77.20.25"
user = "root"
password = "sushma25121998"
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


In [16]:
cities_df = pd.read_sql("cities", con=connection_string)
cities_df

,city_id,city,country,latitude,longitude
0,1,Berlin,Germany,52.520000,13.405000
1,2,Hamburg,Germany,53.550000,10.000000
2,3,Munich,Germany,48.137500,11.575000
3,4,Cologne,Germany,50.936389,6.952778
4,5,Amsterdam,Netherlands,52.372778,4.893611
5,6,Paris,France,48.856667,2.352222


### Getting the "City_id" to the airports dataframe

In [18]:
cities_airports_merged = cities_df.merge(airports_to_db,
                                   left_on = "city",
                                   right_on = "municipalityName",
                                   how="left")

cities_airports_merged

,city_id,city,country,latitude,longitude,icao,name,municipalityName
0,1,Berlin,Germany,52.520000,13.405000,EDDB,Berlin Brandenburg,Berlin
1,2,Hamburg,Germany,53.550000,10.000000,NaN,NaN,NaN
2,3,Munich,Germany,48.137500,11.575000,NaN,NaN,NaN
3,4,Cologne,Germany,50.936389,6.952778,NaN,NaN,NaN
4,5,Amsterdam,Netherlands,52.372778,4.893611,NaN,NaN,NaN
5,6,Paris,France,48.856667,2.352222,LFPB,Paris -Le Bourget,Paris
6,6,Paris,France,48.856667,2.352222,LFPO,Paris -Orly,Paris
7,6,Paris,France,48.856667,2.352222,LFPG,Paris Charles de Gaulle,Paris


In [20]:
airports_df = cities_airports_merged[["icao","name", "city_id"]]

In [22]:
airports_df

,icao,name,city_id
0,EDDB,Berlin Brandenburg,1
1,NaN,NaN,2
2,NaN,NaN,3
3,NaN,NaN,4
4,NaN,NaN,5
5,LFPB,Paris -Le Bourget,6
6,LFPO,Paris -Orly,6
7,LFPG,Paris Charles de Gaulle,6


In [24]:
# Make sure you are working with a copy of the DataFrame, not a view
airports_df = airports_df.loc[:, :].copy()

# Now rename the column
airports_df.rename(columns={"name": "airport_name"}, inplace=True)

In [56]:
airports_df

,icao,Airport_name,city_id
0,EDDB,Berlin Brandenburg,1
1,NaN,NaN,2
2,NaN,NaN,3
3,NaN,NaN,4
4,NaN,NaN,5
5,LFPB,Paris -Le Bourget,6
6,LFPO,Paris -Orly,6
7,LFPG,Paris Charles de Gaulle,6


In [26]:
# Drop rows where 'icao' or 'Airport_name' is None
airports_df_clean = airports_df.dropna(subset=['icao', 'airport_name'])

# Now insert the clean DataFrame into the database
airports_df_clean.to_sql('airports', con=connection_string, if_exists='append', index=False)

4